In [ ]:
__author__ = 'Anderson Carlos Ferreira da Silva'

import sys
import logging
import textwrap
from abc import ABCMeta
from operator import attrgetter
from skmultiflow.core.utils.utils import *
from skmultiflow.classification.base import BaseClassifier

class ADFHoeffdingTree(BaseClassifier):
    
    def __init__(self, nb_threshold=0, nominal_attributes=None):    
         """ADFHoeffdingTree class constructor."""
        super().__init__()        
        self.nb_threshold = nb_threshold
        self.nominal_attributes = nominal_attributes            

    @property
    def nb_threshold(self):
        return self._nb_threshold

    @nb_threshold.setter
    def nb_threshold(self, nb_threshold):
        self._nb_threshold = nb_threshold

    @property
    def nominal_attributes(self):
        return self._nominal_attributes

    @nominal_attributes.setter
    def nominal_attributes(self, nominal_attributes):
        if nominal_attributes is None:
            nominal_attributes = []
            logger.info("No Nominal attributes have been defined, will consider all attributes as numerical")
        self._nominal_attributes = nominal_attributes

    def fit(self, X, y):
        raise NotImplementedError

    def partial_fit(self, X, y):
        """Incrementally trains the model. Train samples (instances) are compossed of X attributes and their
        corresponding targets y.
        Parameters
        ----------
        X: numpy.ndarray of shape (n_samples, n_features)
            Instance attributes.
        y: array_like
            Classes (targets) for all samples in X.
        classes: Not used.
        """
        if y is not None:
            r, _ = get_dimensions(X)
            for i in range(r):
                raise NotImplementedError
    
    def predict(self, X):
        """Predicts the label of the X instance(s)
        Parameters
        ----------
        X: numpy.ndarray of shape (n_samples, n_features)
            Samples for which we want to predict the labels.
        Returns
        -------
        list
            Predicted labels for all instances in X.
        """
        r, _ = get_dimensions(X)
        predictions = []
        for i in range(r):
            raise NotImplementedError
        return predictions

    def predict_proba(self, X):
        raise NotImplementedError
   
    def score(self, X, y):
        raise NotImplementedError

    def get_info(self):
        """Collect information about the Hoeffding Tree configuration.
        Returns
        -------
        string
            Configuration for the Hoeffding Tree.
        """
        description = type(self).__name__ + ': '
        description += 'nb_threshold: {} - '.format(self.nb_threshold)
        description += 'nominal_attributes: {} - '.format(self.nominal_attributes)
        return description